In [1]:
import tweepy, cnfg, sys, os, logging, json

In [2]:
logging.basicConfig(filename='twitter_error_log.log',filemode='w', level=logging.ERROR)

In [3]:
# with open('twitter_error_log.log', 'w'):
#     pass

In [3]:
config = cnfg.load(".twitter_config")
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']

In [4]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,
                 parser=tweepy.parsers.JSONParser()
                )
 
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [5]:
# Original version searched for 'agw', not '#agw', so I got some bad matches that I later removed.
climate_search = ('globalwarming OR (global AND warming) OR #climate OR (climate AND (science OR scientist OR scientists OR alarmist OR alarmists OR change OR realist OR realists OR denial OR denier OR deniers)) OR climatechange OR climaterealists OR #agw')

In [7]:
searchQuery = climate_search

In [8]:
maxTweets = 200 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are reqd, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1L

tweetCount = 0
tweetFiles = 0
msg =  "Downloading max {0} tweets".format(maxTweets)
logging.error(msg)
tweets = []
while 4000*tweetFiles + tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
                
        new_tweets = new_tweets['statuses']
        if len(new_tweets) == 0:
            msg = 'No tweets found'
            logging.error(msg)
            break
        
        tweets.extend(new_tweets)
        tweetCount += len(new_tweets)
        msg = "Downloaded {0} tweets".format(tweetCount + tweetFiles*4000)
        logging.error(msg)
        max_id = new_tweets[-1]['id']
        
        if tweetCount > 4000:
            with open('tweets'+str(tweetFiles)+'.json', 'wb') as outfile:  
                json.dump(tweets, outfile)
            msg = "JSON file saved"
            logging.error(msg)
            tweetFiles += 1
            tweetCount = 0
            del tweets[:]
                
    except tweepy.TweepError as e:
        msg = 'Query failed when max_id equaled {0}: {1}'.format(max_id, e)
        logging.error(msg)

if tweetCount > 0:
    with open('tweets'+str(tweetFiles)+'.json', 'wb') as outfile:  
        json.dump(tweets, outfile)
    msg = "JSON file saved"
    logging.error(msg)
    tweetFiles += 1
    tweetCount = 0

In [46]:
# with open('tweets1.pkl', 'rb') as picklefile:
#     test_tweets = pickle.load(picklefile)

In [64]:
#test_tweets = api.search(q=searchQuery, count=tweetsPerQry)

In [9]:
len(tweets)

200

In [20]:
import requests
from application_only_auth import Client
client = Client(consumer_key, consumer_secret)

In [79]:
response = client.request(
    "https://api.twitter.com/1.1/application/rate_limit_status.json?resources=help,users,search,statuses")

In [80]:
response

{u'rate_limit_context': {u'application': u'2bKlUYBcnpuQwlTjZvxjbiPjF'},
 u'resources': {u'help': {u'/help/configuration': {u'limit': 15,
    u'remaining': 15,
    u'reset': 1489167411},
   u'/help/languages': {u'limit': 15, u'remaining': 15, u'reset': 1489167411},
   u'/help/privacy': {u'limit': 15, u'remaining': 15, u'reset': 1489167411},
   u'/help/settings': {u'limit': 15, u'remaining': 15, u'reset': 1489167411},
   u'/help/tos': {u'limit': 15, u'remaining': 15, u'reset': 1489167411}},
  u'search': {u'/search/tweets': {u'limit': 450,
    u'remaining': 450,
    u'reset': 1489167411}},
  u'statuses': {u'/statuses/lookup': {u'limit': 300,
    u'remaining': 300,
    u'reset': 1489167411},
   u'/statuses/oembed': {u'limit': 180,
    u'remaining': 180,
    u'reset': 1489167411},
   u'/statuses/retweeters/ids': {u'limit': 300,
    u'remaining': 300,
    u'reset': 1489167411},
   u'/statuses/retweets/:id': {u'limit': 300,
    u'remaining': 300,
    u'reset': 1489167411},
   u'/statuses/show

In [44]:
# with open('tweets1.json') as json_file:  
#     test = json.load(json_file)